## RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [5]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
  """Process all PDF files in a directory"""
  all_documents = []
  pdf_dir = Path(pdf_directory)
  # Find all PDF files recursively
  pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
  print(f"Found {len(pdf_files)} PDF files to process")

  for pdf_file in pdf_files:
    print(f"\nProcessing: {pdf_file.name}")
    try:
      loader = PyPDFLoader(str(pdf_file))
      documents = loader.load()
      
      # Add source information to metadata
      for doc in documents:
          doc.metadata['source_file'] = pdf_file.name
          doc.metadata['file_type'] = 'pdf'
      
      all_documents.extend(documents)
      print(f"✓ Loaded {len(documents)} pages")
        
    except Exception as e:
      print(f"✗ Error: {e}")

  print(f"\nTotal documents loaded: {len(all_documents)}")
  return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: Machine Learning Basics.pdf
✓ Loaded 1 pages

Processing: Python Programming Introduction.pdf
✓ Loaded 1 pages

Total documents loaded: 2


In [6]:
all_pdf_documents

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-10-21T13:51:10+00:00', 'author': 'aayush', 'moddate': '2025-10-21T13:51:10+00:00', 'source': '../data/pdf_files/Machine Learning Basics.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Machine Learning Basics.pdf', 'file_type': 'pdf'}, page_content='Machine Learning Basics \n \nMachine learning is a subset of artificial intelligence that enables systems to learn and \nimprove \nfrom experience without being explicitly programmed. It focuses on developing \ncomputer programs \nthat can access data and use it to learn for themselves. \n \nTypes of Machine Learning: \n1. Supervised Learning: Learning with labeled data \n2. Unsupervised Learning: Finding patterns in unlabeled data \n3. Reinforcement Learning: Learning through rewards and penalties \n \nApplications include image recognition, speech processing, and recommendation \nsystems.'),
 Document(metadata={'producer': 'PyP

In [7]:
### Text splitting get into chunks

def split_documents(documents, chunk_size = 1000, chunk_overlap = 200):
  """Split documents into smaller chunks for better RAG performance"""
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap,
      length_function = len,
      separators = ["\n\n", "\n", " ", ""]
  )
  split_docs = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
  
  # Show example of a chunk
  if split_docs:
      print(f"\nExample chunk:")
      print(f"Content: {split_docs[0].page_content[:200]}...")
      print(f"Metadata: {split_docs[0].metadata}")
  
  return split_docs

In [8]:
chunks=split_documents(all_pdf_documents)
chunks

Split 2 documents into 2 chunks

Example chunk:
Content: Machine Learning Basics 
 
Machine learning is a subset of artificial intelligence that enables systems to learn and 
improve 
from experience without being explicitly programmed. It focuses on develo...
Metadata: {'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-10-21T13:51:10+00:00', 'author': 'aayush', 'moddate': '2025-10-21T13:51:10+00:00', 'source': '../data/pdf_files/Machine Learning Basics.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Machine Learning Basics.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-10-21T13:51:10+00:00', 'author': 'aayush', 'moddate': '2025-10-21T13:51:10+00:00', 'source': '../data/pdf_files/Machine Learning Basics.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Machine Learning Basics.pdf', 'file_type': 'pdf'}, page_content='Machine Learning Basics \n \nMachine learning is a subset of artificial intelligence that enables systems to learn and \nimprove \nfrom experience without being explicitly programmed. It focuses on developing \ncomputer programs \nthat can access data and use it to learn for themselves. \n \nTypes of Machine Learning: \n1. Supervised Learning: Learning with labeled data \n2. Unsupervised Learning: Finding patterns in unlabeled data \n3. Reinforcement Learning: Learning through rewards and penalties \n \nApplications include image recognition, speech processing, and recommendation \nsystems.'),
 Document(metadata={'producer': 'PyP

### Embedding And vectorStoreDB


In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [11]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [12]:
chunks

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-10-21T13:51:10+00:00', 'author': 'aayush', 'moddate': '2025-10-21T13:51:10+00:00', 'source': '../data/pdf_files/Machine Learning Basics.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Machine Learning Basics.pdf', 'file_type': 'pdf'}, page_content='Machine Learning Basics \n \nMachine learning is a subset of artificial intelligence that enables systems to learn and \nimprove \nfrom experience without being explicitly programmed. It focuses on developing \ncomputer programs \nthat can access data and use it to learn for themselves. \n \nTypes of Machine Learning: \n1. Supervised Learning: Learning with labeled data \n2. Unsupervised Learning: Finding patterns in unlabeled data \n3. Reinforcement Learning: Learning through rewards and penalties \n \nApplications include image recognition, speech processing, and recommendation \nsystems.'),
 Document(metadata={'producer': 'PyP

In [13]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 2 texts...


Batches: 100%|██████████| 1/1 [00:03<00:00,  3.28s/it]

Generated embeddings with shape: (2, 384)
Adding 2 documents to vector store...
Successfully added 2 documents to vector store
Total documents in collection: 2


### Retriever Pipeline From VectorStore


In [16]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)
rag_retriever

In [17]:
rag_retriever.retrieve("What are types of Machine Learning?")

Retrieving documents for query: 'What are types of Machine Learning?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.77s/it]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


[{'id': 'doc_8d335dc9_0',
  'content': 'Machine Learning Basics \n \nMachine learning is a subset of artificial intelligence that enables systems to learn and \nimprove \nfrom experience without being explicitly programmed. It focuses on developing \ncomputer programs \nthat can access data and use it to learn for themselves. \n \nTypes of Machine Learning: \n1. Supervised Learning: Learning with labeled data \n2. Unsupervised Learning: Finding patterns in unlabeled data \n3. Reinforcement Learning: Learning through rewards and penalties \n \nApplications include image recognition, speech processing, and recommendation \nsystems.',
  'metadata': {'moddate': '2025-10-21T13:51:10+00:00',
   'creator': 'Microsoft Word',
   'file_type': 'pdf',
   'page': 0,
   'source_file': 'Machine Learning Basics.pdf',
   'content_length': 583,
   'total_pages': 1,
   'producer': 'PyPDF',
   'author': 'aayush',
   'doc_index': 0,
   'creationdate': '2025-10-21T13:51:10+00:00',
   'source': '../data/pdf_

In [18]:
rag_retriever.retrieve("What are key features of Python?")

Retrieving documents for query: 'What are key features of Python?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.32it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


[{'id': 'doc_3dce3479_1',
  'content': 'Python Programming Introduction \n \nPython is a high-level, interpreted programming language known for its simplicity and \nreadability. \nCreated by Guido van Rossum and first released in 1991, Python has become one of the \nmost popular \nprogramming languages in the world. \n \nKey Features: \n- Easy to learn and use \n- Extensive standard library \n- Cross-platform compatibility \n- Strong community support \n \nPython is widely used in web development, data science, artificial intelligence, and \nautomation.',
  'metadata': {'page_label': '1',
   'file_type': 'pdf',
   'author': 'aayush',
   'total_pages': 1,
   'source': '../data/pdf_files/Python Programming Introduction.pdf',
   'producer': 'PyPDF',
   'creationdate': '2025-10-21T13:50:50+00:00',
   'creator': 'Microsoft Word',
   'moddate': '2025-10-21T13:50:50+00:00',
   'content_length': 504,
   'source_file': 'Python Programming Introduction.pdf',
   'page': 0,
   'doc_index': 1},
  '

### Integration Vectordb Context pipeline With LLM output

In [35]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [36]:
answer=rag_simple("What is Machine Learning?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is Machine Learning?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
Machine Learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed.


### Enhanced RAG Pipeline Features

In [37]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Types of Machine learning", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Types of Machine learning'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]



Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
Answer: There are three main types of machine learning:

1. Supervised Learning
2. Unsupervised Learning
3. Reinforcement Learning
Sources: [{'source': 'Machine Learning Basics.pdf', 'page': 0, 'score': 0.5012401044368744, 'preview': 'Machine Learning Basics \n \nMachine learning is a subset of artificial intelligence that enables systems to learn and \nimprove \nfrom experience without being explicitly programmed. It focuses on developing \ncomputer programs \nthat can access data and use it to learn for themselves. \n \nTypes of Machin...'}]
Confidence: 0.5012401044368744
Context Preview: Machine Learning Basics 
 
Machine learning is a subset of artificial intelligence that enables systems to learn and 
improve 
from experience without being explicitly programmed. It focuses on developing 
computer programs 
that can access data and use it to learn for themselves. 
 
Types of Machin
Answer: There are t

In [39]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is machine learning", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is machine learning'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
Machine Learning Basics 
 
Machine learning is a subset of artificial intelligence that enarning Basics 
 
Machine learning is a subset of artificial intelligence that enables systems to learn and 
improve 
from experience without being explicitly probles systems to learn and 
improve 
from experience without being explicitly programmed. It focuses on developing 
computer programs 
that can access data and ugrammed. It focuses on developing 
computer programs 
that can access data and u

se it to learn for themselves. 
 
Types of Machine Learning: 
1. Supervised Learning: Learning with labeled data 
2. Unsupervised Learning: Finding patterns in ning: Learning with labeled data 
2. Unsupervised Learning: Finding patterns in unlabeled data 
3. Reinforcement Learning: Learning through rewards and penaltieunlabeled data 
3. Reinforcement Learning: Learning through rewards and penalties 
 
Applications include image recognition, speech processing, and recommendatis 
 
Applications include image recognition, speech processing, and recommendation 
systems.

Question: what is machine learning

Answer:on 
systems.

Question: what is machine learning

Answer:


Final Answer: Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed.

Citations:
[1] Machine Learning Basics.pdf (page 0)
Summary: Machine learning is a subset of artificial intelligence that allows systems to learn and improve